In [61]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import json
import folium
import random
import itertools
from geopy.geocoders import Photon
from geopy.geocoders import Nominatim

np.random.seed(0)

In [ ]:
# definição dos grupos de filiais pra facilitar a vida

df = pd.read_csv("dados_simulacao/distancias_todas_combinacoes.csv")

CD = 'CENTRO DE DISTRIBUICAO'

todas_com_cd = df

todas_sem_cd = df[(df['Filial_A'] != CD) 
                      & (df['Filial_B'] != CD)]

rj_com_cd = df[(df['Estado_A'] == 'RJ')|(df['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ

rj_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'RJ')|(todas_sem_cd['Estado_B'] == 'RJ')]  # uniao de E_A e E_B == RJ \{filial = CD}

sp_sem_cd = todas_sem_cd[(todas_sem_cd['Estado_A'] == 'SP')|(todas_sem_cd['Estado_B'] == 'SP')] # uniao de E_A e E_B == SP

cd_sp = df[((df['Estado_A'] == 'SP') & (df['Filial_B'] == CD))
           | ((df['Estado_B'] == 'SP') & (df['Filial_A'] == CD))]
sp_com_cd = pd.concat((cd_sp, sp_sem_cd), ignore_index=True)  # uniao de E_A e E_B == SP  +  {A = CD e E_B = SP}

lojas_A = todas_sem_cd[['Estado_A', 'Filial_A']].rename(columns={'Estado_A': 'Estado', 'Filial_A': 'Filial'})
lojas_B = todas_sem_cd[['Estado_B', 'Filial_B']].rename(columns={'Estado_B': 'Estado', 'Filial_B': 'Filial'})

um_de_cada = pd.concat([lojas_A, lojas_B]) \
                     .drop_duplicates() \
                     .sort_values(by=['Estado', 'Filial']) \
                     .drop_duplicates(subset=['Estado']) \
                     .reset_index(drop=True)

estado_sem_cd = todas_sem_cd[(todas_sem_cd['Filial_A'].isin(um_de_cada['Filial'].tolist())) &
                             (todas_sem_cd['Filial_B'].isin(um_de_cada['Filial'].tolist()))]

cd_filiais = df[(df['Filial_A'] == CD) & (df['Filial_B'].isin(um_de_cada['Filial'].tolist()))]

estado_com_cd = pd.concat([estado_sem_cd, cd_filiais], ignore_index=True)

#### Utilize uma das variáveis para a simulação:

* `todas_sem_cd` para utilizar todas as filiais do Brasil.
* `todas_com_cd` para utilizar todas as filiais do Brasil e o Centro de Distribuição.
* `rj_sem_cd` para utilizar todas as filiais do Rio de Janeiro.
* `rj_com_cd` para utilizar todas as filiais do Rio de Janeiro e o Centro de Distribuição.
* `sp_sem_cd` para utilizar todas as filiais de São Paulo.
* `sp_com_cd` para utilizar todas as filiais de São Paulo e o Centro de Distribuição.
* `estado_sem_cd` para utilizar uma filial e cada estado.
* `estado_com_cd` para utilizar uma filial e cada estado e o Centro de Distribuição.

#### Não esqueça de especificar se o CD está incluso ou não (True/False)

In [ ]:
# escolha das variáveis

filiais = todas_sem_cd # qual conjunto de variáveis usar 
cd = False # booleano se considera ou não o CD no grafo

custo_km = 0.5  # custo do frete em reais por km
leadtime_km = 1 #
estoque = 1
max_envio = np.floor(n-(8/10)*estoque_)

In [ ]:
def get_distancia(origem, destino, filiais):
    """
    Consulta a distância entre duas filiais no DataFrame.
    Esta função é bidirecional (procura A->B e B->A).
    """

    dist_df = filiais[
        ((filiais['Filial_A'] == origem) & (filiais['Filial_B'] == destino)) |
        ((filiais['Filial_A'] == destino) & (filiais['Filial_B'] == origem))]
        
    return dist_df['Distancia_km'].values[0]

In [56]:
# Criar o  grafo